# Data Sharing with Encryption - Delentture side

### Recommended 

Create conda enviroment to avoid conflicts with packages.

### Installs

In [ ]:
!pip uninstall -y pycrypto
!pip install pycryptodome
!pip install seaborn

### Imports & W/R byte functions

In [ ]:
import seaborn as sb
from Crypto.Cipher import AES
from Crypto import Random
from cryptography.hazmat.backends import default_backend
import socket

HOST = "127.0.0.1"  # The server's hostname or IP address
PORT = 65432  

def write_file(data,filename):
    with open(filename, 'wb') as f:
        f.write(data)
        f.close()

def read_file(filename):
    with open(filename, 'rb') as f:
        data_b = f.read()
        f.close()
    return data_b


### Public Key Exchange

#### DH (Diffie–Hellman)

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat,load_der_public_key

p = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF
g = 2

params_numbers = dh.DHParameterNumbers(p,g)
parameters = params_numbers.parameters(default_backend())

In [ ]:
def dh_key_exchange(params):
    key_B = parameters.generate_private_key() # red
    public_mix_B  = key_B.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo)  #orange
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        public_mix_A = s.recv(1024)
        print(f'Received public key in bytes from Controler {public_mix_A}')
        s.sendall(public_mix_B)
        
    public_mix_A_ser = load_der_public_key(public_mix_A,backend = default_backend())
    shared_key = key_B.exchange(public_mix_A_ser)

    derived_key = HKDF(
        algorithm = hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)

    return derived_key

In [ ]:
dh_key = dh_key_exchange(parameters)
dh_key

**Action #2** -> Run all the upper cells

#### RSA (Rivest–Shamir–Adleman)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

### De-encryption

#### HMAC

In [ ]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import cmac, hashes, hmac
import os

key_1 = b'\xb6\xb3\xe8\xb2\x10\x07\xf9\x82\xb6\xfb\xdc\xf5\xc2\xcd\x1d\xd6I\x83\xd6\xf0)w\xaawV8\x93SH&C\xf7'
key_2 = b'$C&F)J@NcRfUjXnZ'

In [ ]:
h = hmac.HMAC(key_2, hashes.SHA256())
h.update(ciphertext)
h.verify(signature)

#### AES (ECB)

In [ ]:
def decrypt_image_ecb(data, key, iv=b''):
    
    block_size = AES.block_size
    padding_length = block_size - len(data) % block_size # Calculate padding_length to make len(image array) a multiple of block_size
    data += bytes(padding_length * ".", "UTF-8") # Just an arbitrary padding byte

    mode   = AES.MODE_ECB
    aes    = AES.new(key, mode)

    decrypted_data = aes.decrypt(data) # Your code to decrypt image_array using AES

    return decrypted_data

#### AES (CBC)

### EDA (Exploratory Data Analysis)